In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

In [2]:
import pandas as pd
import numpy as np

import time
import tqdm 
from datetime import datetime

import pickle
import torch
import torch.nn as nn

import shap as shap
#from captum.attr import KernelShap
from captum.attr import IntegratedGradients, Saliency, NoiseTunnel, InputXGradient
from lime import lime_tabular


# load data

In [3]:
#load data
data_train = pd.read_csv('data/compas-scores-train-70.csv')  
data_test = pd.read_csv('data/compas-scores-test-30.csv')  

display(data_train.head())

print('data, train:', data_train.shape)
print('data, test:', data_test.shape)

#split into X, y
X_train = data_train.loc[:, data_train.columns != 'risk']
y_train = data_train['risk']

X_test = data_test.loc[:, data_test.columns != 'risk']
y_test = data_test['risk']


X=X_train
y=y_train
print('----- TRAIN -----')
print('X, shape:', X.shape)
print('y, shape:', y.shape)
print('#class1: ', sum(y), f', prop = {sum(y)/len(y)}')
print('#class0:', sum(y==0), f', prop = {sum(y==0)/len(y)}')

X=X_test
y=y_test
print('----- TEST -----')
print('X, shape:', X.shape)
print('y, shape:', y.shape)
print('#class1: ', sum(y), f', prop = {sum(y)/len(y)}')
print('#class0:', sum(y==0), f', prop = {sum(y==0)/len(y)}')


,age,two_year_recid,priors_count,length_of_stay,c_charge_degree_F,sex_Female,race,risk
0,28,1,6,80,1,0,1,0
1,25,1,9,87,1,0,0,0
2,33,1,5,0,1,1,1,0
3,30,0,0,6,1,0,0,1
4,37,1,0,1,0,0,1,1


data, train: (3455, 8)
data, test: (1482, 8)
----- TRAIN -----
X, shape: (3455, 7)
y, shape: (3455,)
#class1:  2805 , prop = 0.8118668596237337
#class0: 650 , prop = 0.18813314037626627
----- TEST -----
X, shape: (1482, 7)
y, shape: (1482,)
#class1:  1217 , prop = 0.8211875843454791
#class0: 265 , prop = 0.1788124156545209


# load models

In [4]:
#load models

#####logistic regression
model_filename = 'models/model_logistic.pkl'
model_logistic = pickle.load(open(model_filename, 'rb'))

######gradient boosted tree
model_filename = 'models/model_gb.pkl'
model_gb = pickle.load(open(model_filename, 'rb'))

######random forest
model_filename = 'models/model_rf.pkl'
model_rf = pickle.load(open(model_filename, 'rb'))

######FFNN

#module
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, seed=12345):
        super().__init__()
        
        torch.manual_seed(seed)
        
        #variables
        self.input_size = input_size
        self.hidden_size = hidden_size
        #layers architecture
        self.linear_layer1 = nn.Linear(self.input_size, self.hidden_size)
        self.linear_layer2 = nn.Linear(self.hidden_size, self.hidden_size*2)
        self.linear_layer3 = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.linear_layer4 = nn.Linear(self.hidden_size, 1)
        
    def forward(self, inputs):
        out = self.linear_layer1(inputs)
        out = nn.functional.relu(out)
        out = self.linear_layer2(out)
        out = nn.functional.relu(out)
        out = self.linear_layer3(out)
        out = nn.functional.relu(out)
        out = self.linear_layer4(out)
        out = torch.sigmoid(out)
        return out
    
    def predict_proba(self, X):
        X = torch.FloatTensor(X)
        class1_probs = self.forward(X).detach().numpy()
        class0_probs = 1-class1_probs
        return np.hstack((class0_probs, class1_probs))
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)


model_filename = 'models/model_nn.pkl'
model_nn = torch.load(model_filename)


##### NN logistic regression

#module

#create model class
class LogisticRegressionNN(nn.Module):
    def __init__(self, input_size, seed=12345):
        super().__init__()
        
        torch.manual_seed(seed)
        
        #variables
        self.input_size = input_size
        #layers
        self.linear_layer = nn.Linear(self.input_size, 1)
        
    def forward(self, inputs):
        out = self.linear_layer(inputs)
        out = torch.sigmoid(out)
        return out
    
    def predict_proba(self, X):
        X = torch.tensor(X).type(torch.FloatTensor)
        class1_probs = self.forward(X).detach().numpy()
        class0_probs = 1-class1_probs
        return np.hstack((class0_probs, class1_probs))
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)


model_filename = 'models/model_nn_logistic.pkl'
model_nn_logistic = torch.load(model_filename)

# ---------- METHOD 1: kernelshap ----------

In [11]:
#kernelshap function
def explain_kernelshap(predict_fn, instances, filename,
                       background_data, nsamples):
    #run kernelshap
    explainer = shap.KernelExplainer(model=predict_fn, data=background_data)
    shap_values = explainer.shap_values(X=instances, nsamples=nsamples)
    
    #save shap values
    pickle.dump(shap_values, open(filename, 'wb'))

    return shap_values


In [12]:
#predict functions --> input for explain_kernelshap

# predict_fn: predicts probability of class1
#     in: instances, 2D np.array, n(=#datapoints) x p(=#features)
#     out: predictions, 1D np.array, n


def predict_fn_logistic(instances):
    return model_logistic.predict_proba(instances)[:, 1]

def predict_fn_gb(instances):
    return model_gb.predict_proba(instances)[:, 1]

def predict_fn_rf(instances):
    return model_rf.predict_proba(instances)[:, 1]

def predict_fn_nn(instances):
    return model_nn.predict_proba(instances)[:, 1]

def predict_fn_nn_logistic(instances):
    return model_nn_logistic.predict_proba(instances)[:, 1]


In [13]:
#number instances to explain
n= X_test.shape[0] #10

#general arguments
instances = X_test.values[0:n, :]

#shap arguments
background_data = X_test.values
nsamples = 2**7


model_names = ['logistic', 'gb', 'rf', 'nn', 'nn_logistic']
predict_fns = {'logistic': predict_fn_logistic, 
               'gb': predict_fn_gb, 
               'rf': predict_fn_rf, 
               'nn': predict_fn_nn,
               'nn_logistic': predict_fn_nn_logistic}
filenames_ks = {'logistic': 'explanations/expl_kernelshap_logistic.pkl', 
                'gb': 'explanations/expl_kernelshap_gb.pkl', 
                'rf': 'explanations/expl_kernelshap_rf.pkl', 
                'nn': 'explanations/expl_kernelshap_nn.pkl',
                'nn_logistic': 'explanations/expl_kernelshap_nn_logistic.pkl'}

In [14]:
#explain all models using kernelshap
expl_shap = {m: explain_kernelshap(predict_fns[m], instances, filenames_ks[m], background_data, nsamples) 
             for m in model_names}

Using 1482 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1482 [00:00<?, ?it/s]

Using 1482 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1482 [00:00<?, ?it/s]

Using 1482 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1482 [00:00<?, ?it/s]

Using 1482 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1482 [00:00<?, ?it/s]

Using 1482 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1482 [00:00<?, ?it/s]

In [15]:
#load kernelshap explanations

model_names = ['logistic', 'gb', 'rf', 'nn', 'nn_logistic']

filenames_ks = {'logistic': 'explanations/expl_kernelshap_logistic.pkl', 
                'gb': 'explanations/expl_kernelshap_gb.pkl', 
                'rf': 'explanations/expl_kernelshap_rf.pkl', 
                'nn': 'explanations/expl_kernelshap_nn.pkl',
                'nn_logistic': 'explanations/expl_kernelshap_nn_logistic.pkl'}


expl_shap = {m: pickle.load(open(filenames_ks[m], 'rb')) for m in model_names}

# ---------- METHOD 2: vanilla gradient ----------

In [5]:
def explain_vanilla_grad(model, instances, filename):
    model.zero_grad()
    method = Saliency(model)
    attr = method.attribute(instances)
    
    attr_np = attr.numpy()
    pickle.dump(attr_np, open(filename, 'wb'))
    return attr_np

## NN

In [6]:
#parameters
n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

model = model_nn
filename = 'explanations/expl_vanillagrad_nn.pkl'

#run explanation 
expl_vanillagrad_nn = explain_vanilla_grad(model, instances, filename)

#load explanation
expl_vanillagrad_nn = pickle.load(open(filename, 'rb'))

Input Tensor 0 did not already require gradients, required_grads has been set automatically.


## logistic

In [7]:
#parameters
n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

model = model_nn_logistic
filename = 'explanations/expl_vanillagrad_nn_logistic.pkl'

#run explanation 
expl_vanillagrad_nn_logistic = explain_vanilla_grad(model, instances, filename)

#load explanation
expl_vanillagrad_nn_logistic = pickle.load(open(filename, 'rb'))

# ---------- METHOD 3: GRADIENT*INPUT ----------

In [8]:
def explain_gradtinput(model, instances, filename):
    model.zero_grad()
    method = InputXGradient(model)
    attr = method.attribute(instances)
    
    attr_np = attr.detach().numpy()
    pickle.dump(attr_np, open(filename, 'wb'))
    return attr_np

## NN

In [9]:
#parameters
n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

model = model_nn
filename = 'explanations/expl_gradtinput_nn.pkl'

#run explanation
expl_gradtinput_nn = explain_gradtinput(model, instances, filename)

#load explanation
expl_gradtinput_nn = pickle.load(open(filename, 'rb'))

## logistic

In [10]:
#parameters
n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

model = model_nn_logistic
filename = 'explanations/expl_gradtinput_nn_logistic.pkl'

#run explanation
expl_gradtinput_nn_logistic = explain_gradtinput(model, instances, filename)

#load explanation
expl_gradtinput_nn_logistic = pickle.load(open(filename, 'rb'))

# ---------- METHOD 4: integrated gradients ----------

In [16]:
def explain_integrated_grad(model, instances, n_steps=50):
    model.zero_grad()
    method = IntegratedGradients(model)
    attr = method.attribute(instances, n_steps=n_steps)
    return attr.numpy()

## explore convergence, NN

In [17]:
def check_convergence(model, instances, nsamples_list, filename, expl_method=['integratedgrad', 'smoothgrad']):
    #dict to store attributions for each sample_size in nsamples_list (sample_size: attributions)
    convergence_attr = {}
    
    for i in nsamples_list:
        
        print(f'nsamples={i}')
        start = time.time()
        print(f'   start: {datetime.now()}')
        
        #run explanation method
        if expl_method=='integratedgrad':
            expl_i = explain_integrated_grad(model, instances, n_steps=i)
        elif expl_method=='smoothgrad':
            expl_i = explain_smoothgrad(model, instances, nt_samples=i)
        #store values
        convergence_attr[i] = expl_i
        
        stop = time.time()
        print(f'   stop: {datetime.now()}')
        print(f'   duration: {(stop-start)/60} min')
        
    #save data
    pickle.dump(convergence_attr, open(filename, 'wb'))

    return convergence_attr
    

In [18]:
#explore convergence, nn --- try different n_steps

#parameters
model = model_nn

n = X_test.shape[0] #10
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500]
expl_method='integratedgrad'
filename=f'convergence/convergence_{expl_method}_nn.pkl'

#check convergence
convergence_integratedgrad_nn = check_convergence(model, instances, nsamples_list, filename, expl_method) #!!!

#load file
convergence_integratedgrad_nn = pickle.load(open(filename, 'rb'))


nsamples=50
   start: 2021-12-12 15:16:56.050523
   stop: 2021-12-12 15:17:06.659217
   duration: 0.17681156794230143 min
nsamples=100
   start: 2021-12-12 15:17:06.659357
   stop: 2021-12-12 15:18:02.262260
   duration: 0.9267150322596233 min
nsamples=200
   start: 2021-12-12 15:18:02.262461
   stop: 2021-12-12 15:23:51.683379
   duration: 5.823681934674581 min
nsamples=400
   start: 2021-12-12 15:23:51.683575
   stop: 2021-12-12 15:57:02.203592
   duration: 33.17533359924952 min
nsamples=600
   start: 2021-12-12 15:57:02.203774
   stop: 2021-12-12 17:19:15.437883
   duration: 82.22056846618652 min
nsamples=800
   start: 2021-12-12 17:19:15.438077
   stop: 2021-12-12 19:48:10.578240
   duration: 148.91900267998378 min
nsamples=1000
   start: 2021-12-12 19:48:10.578435
   stop: 2021-12-12 23:45:58.563329
   duration: 237.7997476498286 min
nsamples=1500
   start: 2021-12-12 23:45:58.564040
   stop: 2021-12-13 08:55:44.807521
   duration: 549.770723982652 min


In [19]:
#explore convergence, nn --- plot


In [20]:
#explore convergence, nn --- pick and save explanation


## explore convergence, logistic

In [21]:
#explore convergence, logistic

#explore convergence, nn --- try different n_steps

#parameters
model = model_nn_logistic

n = X_test.shape[0] #10
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500]
expl_method='integratedgrad'
filename=f'convergence/convergence_{expl_method}_nn_logistic.pkl'



#check convergence
convergence_integratedgrad_nn_logistic = check_convergence(model, instances, nsamples_list, filename, expl_method) #!!!

#load file
convergence_integratedgrad_nn_logistic = pickle.load(open(filename, 'rb'))



nsamples=50
   start: 2021-12-13 08:55:44.842723
   stop: 2021-12-13 08:55:51.054035
   duration: 0.10352184772491455 min
nsamples=100
   start: 2021-12-13 08:55:51.054209
   stop: 2021-12-13 08:56:39.916304
   duration: 0.814368216196696 min
nsamples=200
   start: 2021-12-13 08:56:39.916427
   stop: 2021-12-13 09:02:19.531799
   duration: 5.660256167252858 min
nsamples=400
   start: 2021-12-13 09:02:19.531962
   stop: 2021-12-13 09:34:34.884903
   duration: 32.255882330735524 min
nsamples=600
   start: 2021-12-13 09:34:34.885297
   stop: 2021-12-13 10:55:08.090098
   duration: 80.55341333548228 min
nsamples=800
   start: 2021-12-13 10:55:08.090277
   stop: 2021-12-13 13:08:25.663816
   duration: 133.29289159377416 min
nsamples=1000
   start: 2021-12-13 13:08:25.664945
   stop: 2021-12-13 17:06:35.754324
   duration: 238.16815556287764 min
nsamples=1500
   start: 2021-12-13 17:06:35.755503
   stop: 2021-12-14 02:05:15.161910
   duration: 538.6567727168401 min


# ---------- METHOD 5: smoothgrad ----------

In [22]:
def explain_smoothgrad(model, instances, nt_samples=5):
    model.zero_grad()
    method = NoiseTunnel(Saliency(model))
    attr = method.attribute(instances, nt_type='smoothgrad', nt_samples=nt_samples)
    return attr.numpy()

## NN

In [23]:
#smoothgrad
#explore convergence, nn --- try different n_steps

#parameters
model = model_nn

n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500]
expl_method='smoothgrad'
filename=f'convergence/convergence_{expl_method}_nn.pkl'

#check convergence
convergence_smoothgrad_nn = check_convergence(model, instances, nsamples_list, filename, expl_method) #!!!

#load file
convergence_smoothgrad_nn = pickle.load(open(filename, 'rb'))


nsamples=50
   start: 2021-12-14 02:05:15.201352
   stop: 2021-12-14 02:05:21.405546
   duration: 0.10340323448181152 min
nsamples=100
   start: 2021-12-14 02:05:21.405654
   stop: 2021-12-14 02:06:12.456221
   duration: 0.8508427500724792 min
nsamples=200
   start: 2021-12-14 02:06:12.456395
   stop: 2021-12-14 02:12:03.776269
   duration: 5.855331218242645 min
nsamples=400
   start: 2021-12-14 02:12:03.776445
   stop: 2021-12-14 02:45:25.377062
   duration: 33.36001027027766 min
nsamples=600
   start: 2021-12-14 02:45:25.377232
   stop: 2021-12-14 04:08:34.738138
   duration: 83.15601507027944 min
nsamples=800
   start: 2021-12-14 04:08:34.738513
   stop: 2021-12-14 06:40:11.506270
   duration: 151.61279593308765 min
nsamples=1000
   start: 2021-12-14 06:40:11.507818
   stop: 2021-12-14 10:39:27.175391
   duration: 239.26112620035806 min
nsamples=1500
   start: 2021-12-14 10:39:27.175585
   stop: 2021-12-14 19:34:24.491990
   duration: 534.955272646745 min


## logistic

In [ ]:
#smoothgrad
#explore convergence, nn --- try different n_steps

#parameters
model = model_nn_logistic

n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500]
expl_method='smoothgrad'
filename=f'convergence/convergence_{expl_method}_nn_logistic.pkl'

#check convergence
convergence_smoothgrad_nn_logistic = check_convergence(model, instances, nsamples_list, filename, expl_method) #!!!

#load file
convergence_smoothgrad_nn_logistic = pickle.load(open(filename, 'rb'))


nsamples=50
   start: 2021-12-14 19:34:24.546553
   stop: 2021-12-14 19:34:30.729483
   duration: 0.10304881334304809 min
nsamples=100
   start: 2021-12-14 19:34:30.729603
   stop: 2021-12-14 19:35:08.190801
   duration: 0.624353297551473 min
nsamples=200
   start: 2021-12-14 19:35:08.190976
   stop: 2021-12-14 19:40:59.273244
   duration: 5.85137110153834 min
nsamples=400
   start: 2021-12-14 19:40:59.273412
   stop: 2021-12-14 20:14:07.712701
   duration: 33.14065479834874 min
nsamples=600
   start: 2021-12-14 20:14:07.712903


# ---------- METHOD 6: lime ----------

In [5]:
def explain_lime(predict_fn, instances, training_data, cat_idxs, n_samples, filename,
                 mode=['classification', 'regression'], seed=12345):
    #create lime explainer
    np.random.seed(seed)
    explainer = lime_tabular.LimeTabularExplainer(training_data=training_data, 
                                                  mode=mode, 
                                                  discretize_continuous=False,
                                                  sample_around_instance=True,
                                                  random_state=seed, 
                                                  categorical_features=cat_idxs)
    #explain each data point in 'instances'
    exps = []
    num_feat = instances.shape[1]
    for x in instances:
        exp = explainer.explain_instance(x, 
                                         predict_fn=predict_fn,
                                         num_samples=n_samples,
                                         num_features=num_feat).local_exp[1]
        #format explanations
        exp = sorted(exp, key=lambda tup: tup[0])
        exp = [t[1] for t in exp]
        exps.append(exp)
    
    #save explanations
    exps = np.array(exps)
    pickle.dump(exps, open(filename, 'wb'))
    
    return exps #n x p (same dimensions as 'instances')

In [6]:
def check_convergence_lime(predict_fn, instances, training_data, cat_idxs, filename_lime, mode,
                           nsamples_list, filename_convergence):
    #dict to store attributions for each sample_size in nsamples_list (sample_size: attributions)
    convergence_attr = {}
    
    for i in nsamples_list:
        
        print(f'nsamples={i}')
        start = time.time()
        print(f'   start: {datetime.now()}')
        
        #run explanation method
        expl_i = explain_lime(predict_fn, instances, training_data, cat_idxs, 
                              n_samples=i, filename=f'{filename_lime}_n{i}.pkl', mode=mode)
        #store values
        convergence_attr[i] = expl_i
        
        stop = time.time()
        print(f'   stop: {datetime.now()}')
        print(f'   duration: {(stop-start)/60} min')
        
    #save data
    pickle.dump(convergence_attr, open(filename_convergence, 'wb'))

    return convergence_attr
    

In [7]:
#dictionaries
model_names = ['logistic', 'gb', 'rf', 'nn', 'nn_logistic']
predict_fns_2classes = {'logistic': model_logistic.predict_proba, 
               'gb': model_gb.predict_proba, 
               'rf': model_rf.predict_proba, 
               'nn': model_nn.predict_proba,
               'nn_logistic': model_nn_logistic.predict_proba}


#arguments that are constant
n = X_test.shape[0] #10
instances = X_test.values[0:n, :]
training_data = X_train.values

cat_vars = ['two_year_recid', 'c_charge_degree_F', 'sex_Female', 'race']
cat_idxs = [list(X_train.columns).index(var) for var in cat_vars]

mode = 'classification'
nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500, 2000, 2500, 3000]


In [8]:
#for each model
for m in range(0, len(model_names)):
    model = model_names[m]
    print(f'******convergence analysis for {model}******')
    
    #run convergence analysis
    filename_lime=f'explanations/expl_lime_{model}'
    filename_convergence=f'convergence/convergence_lime_{model}.pkl'
    _ = check_convergence_lime(predict_fn=predict_fns_2classes[model], 
                           instances=instances, 
                           training_data=training_data, 
                           cat_idxs=cat_idxs, 
                           filename_lime=filename_lime, 
                           mode=mode,
                           nsamples_list=nsamples_list, 
                           filename_convergence=filename_convergence)
    

******convergence analysis for logistic******
nsamples=50
   start: 2021-12-14 20:59:11.698089
   stop: 2021-12-14 20:59:16.063180
   duration: 0.07275149822235108 min
nsamples=100
   start: 2021-12-14 20:59:16.063308
   stop: 2021-12-14 20:59:20.484128
   duration: 0.07368031342824301 min
nsamples=200
   start: 2021-12-14 20:59:20.484275
   stop: 2021-12-14 20:59:25.247080
   duration: 0.07938005526860555 min
nsamples=400
   start: 2021-12-14 20:59:25.247230
   stop: 2021-12-14 20:59:29.698626
   duration: 0.07418991724650065 min
nsamples=600
   start: 2021-12-14 20:59:29.698822
   stop: 2021-12-14 20:59:34.903023
   duration: 0.08673668305079142 min
nsamples=800
   start: 2021-12-14 20:59:34.903173
   stop: 2021-12-14 20:59:40.160018
   duration: 0.08761406342188517 min
nsamples=1000
   start: 2021-12-14 20:59:40.160198
   stop: 2021-12-14 20:59:45.553314
   duration: 0.08988524675369262 min
nsamples=1500
   start: 2021-12-14 20:59:45.553473
   stop: 2021-12-14 20:59:51.206601
   dur